In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class Model_Evaluation_Config:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file: Path

In [4]:
from src.brain_tumor_classifier.constant import *
def create_directories(file_path):
    pass
def read_yaml(file_path):
    pass
class Configuration_Manager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH,schema_path=SCHEMA_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(config_path)
        self.schema = read_yaml(config_path)
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self)->Model_Evaluation_Config:
        config = self.config.Model_Evaluation
        create_directories([config.root_dir])
        model_evaluation_config = Model_Evaluation_Config(root_dir=config.root_dir,test_data_path=config.test_data_path,model_path=config.model_path,metric_file=config.metric_file)
        return model_evaluation_config    

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from src.brain_tumor_classifier.utils.common import load_binary_file
TENSOR_SIZE = (224,224,3)

class Model_Evaluation:
    def __init__(self,model_evaluation_config: Model_Evaluation_Config):
        self.config = model_evaluation_config
    def evaluate_models(self,models):
        best_model = models[0]
        best_model_accuracy = models[0].history['val_accuracy']
        for i, model in enumerate(models):
            # Plot and save loss
            plt.plot(model.history['loss'], label='Training Loss')
            plt.plot(model.history['val_loss'], label='Validation Loss')
            plt.legend()
            plt.savefig(f'{self.config.metric_file}/model_{i}_Loss_Val_Loss.png')
            plt.close()

            # Plot and save accuracy
            plt.plot(model.history['accuracy'], label='Training Accuracy')
            plt.plot(model.history['val_accuracy'], label='Validation Accuracy')
            plt.legend()
            plt.savefig(f'{self.config.metric_file}/model_{i}_Accuracy_Val_Accuracy.png')
            plt.close()

            # Update best model if necessary
            if model.history['val_accuracy'] > best_model_accuracy:
                best_model_accuracy = model.history['val_accuracy']
                best_model = model

        return best_model, best_model_accuracy
    
    def initiate_mode_evaluation(self):
        resnet_model  = load_binary_file(os.path.join(self.config.model_path,'resnet_model.h5'))
        vgg_model  = load_binary_file(os.path.join(self.config.model_path,'vgg_model.h5'))

        best_model,best_model_accuracy = self.evaluate_models(models=[vgg_model,resnet_model])
        logging.info('Finished Evaluating Models, Outputs are saved in CWD')
        logging.info(f'Best Model found: {best_model} with Validation Accuracy {best_model_accuracy}')
        return (best_model,best_model_accuracy)

In [ ]:
try:
    config = Configuration_Manager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = Model_Evaluation(model_evaluation_config)
    model_evaluation.initiate_model_evaluation()
except Exception as e:
    raise e